In [33]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM5 - USB-SERIAL CH340 (COM5)
COM4 - USB Serial Port (COM4)


In [35]:
import serial
import time
ser = serial.Serial('COM5', 115200, timeout=10)

In [31]:
ser.close()

In [536]:
#test echo (lab2)
ser.write(b'ABCD EFGHI J')
time.sleep(.001)
print(ser.read(12))
#works again after ser close + reset on pynq -> check yusuf testbench for mismatch rx and tx

b'ABCD EFGHI J'


In [37]:
#test mp adder (lab3) (problem: received missing first bit + small bit flips)
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

res = A + B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

if res2==res:
    print("Result received correctly!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))

A     =  0x9d18fa46b292e03f3c4e26f95e5b2cd83bc6df3a3aaec0c1680856b1a1b669e6470e5c9f136281c0113506dd89993d1c3908415f795652de6c37255d2c45b526
B     =  0xb2afe4948c4b520656aac603ab74ac578782d68b071361c8f7eb3b29ed7cc95d5f3fed9da9b9d89023e59841a348596a69495744fdf5188a62c35410352e3514
A + B =  0x14fc8dedb3ede324592f8ecfd09cfd92fc349b5c541c2228a5ff391db8f333343a64e4a3cbd1c5a50351a9f1f2ce19686a25198a4774b6b68cefa796d6173ea3a
Result INCORRECT!
Expected   =  0x14fc8dedb3ede324592f8ecfd09cfd92fc349b5c541c2228a5ff391db8f333343a64e4a3cbd1c5a50351a9f1f2ce19686a25198a4774b6b68cefa796d6173ea3a
Received   =  0x4fc8deda3ede324692f8ecfd09cfd92fc349b5c541c2228a5ff491db8f333343ed4e4a3cbd1b5a50351a9f1e2ce29687a25298a4774b6b69cefa796d6173ea3a
